In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


Reading a TTree in the same way as during previous cases.

In [2]:
file = r.TFile("events2W.root","READ")
tree = file.Get("tree") # Getting the tree 

Warning in <TFile::Init>: file events2W.root probably not closed, trying to recover
Info in <TFile::Recover>: events2W.root, recovered key TTree:tree at address 19858
Warning in <TFile::Init>: successfully recovered 1 keys


In [3]:
# Initializing 4-element array to store 4-vectors
w1Arr = np.zeros(4)
w2Arr = np.zeros(4)
jet1Arr = np.zeros(4)
jet2Arr = np.zeros(4)
eArr = np.zeros(4)
betaArr = np.zeros(4)
nArr = np.zeros(4)

In [4]:
# Taking already filled branches from the file
# Now we need to add another branch for neutrino
tree.SetBranchAddress("w1Boson", w1Arr)
tree.SetBranchAddress("w2Boson", w2Arr)
tree.SetBranchAddress("jet1", jet1Arr)
tree.SetBranchAddress("jet2", jet2Arr)
tree.SetBranchAddress("electron", eArr)
tree.SetBranchAddress("beta", betaArr)

# Initializing branch for neutrino components
tree.Branch("neutrino",nArr,"neArray[4]/D")

In [5]:
%%cpp -d   

void fcn(int &npar, double *gin, double &f, double *par, int iflag){
    double pTSq = par[0];
    double pTotSq = par[1];
    double ePz = par[2];
    double nPz = par[3];

    double wMass = TMath::Sqrt(2*pTSq + 2 * TMath::Sqrt(pTotSq) * TMath::Sqrt(pTSq + pow(nPz,2)) - 2 * ePz * nPz);

    f = pow(wMass-80.3,2);
};

double minimization_func(double *wArr, double *elArr){
    /*  Input:
        wArr ...... W boson four-vector as an 4-element array [Px,Py,Pz,E]
        elArr...... Electron four-vector as an 4-element array [Px,Py,Pz,E]
        
        Output:
        nPz ....... float value of anti neutrino Pz
    */
    double ePx = elArr[0];
    double ePy = elArr[1];
    double ePz = elArr[2];
    
    double wPz = wArr[2]; 
    
    double initVal;
    
    if(abs(ePz - wPz) > 1){
        initVal = -(ePz - wPz)+1;
    } else {
        initVal = -ePz + 0.02;  
    }
    
    double pTSqEl = pow(ePx,2) + pow(ePy,2);
    double pTotSqEl = pTSqEl + pow(ePz,2);  
    
    TMinuit *gMinuit = new TMinuit(4);
    gMinuit->SetFCN(fcn);
    gMinuit->DefineParameter(0, "pTotSqEl", pTSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(1, "pTSqEl", pTotSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(2, "ePz", ePz, 1., 0., 0.);
    gMinuit->DefineParameter(3, "nPz", initVal, 0.05, 0., 0.);
    gMinuit->FixParameter(0);
    gMinuit->FixParameter(1);
    gMinuit->FixParameter(2);
    gMinuit->Command("MIGRAD");

    double nPz, nPzErr;
    gMinuit->GetParameter(3, nPz, nPzErr);
    return nPz;

};

Iterating over `n` number of entries and calculating the components of anti-neutrino four-vector and filling them into its respective branch.

In [6]:
n = tree.GetEntries() # Number of entries in tree (number of different events)

for i in range(n):
    tree.GetEntry(i)
    ePx = tree.GetListOfLeaves().At(4).GetValue(0)
    ePy = tree.GetListOfLeaves().At(4).GetValue(1)
    jet1Px = tree.GetListOfLeaves().At(2).GetValue(0)
    jet1Py = tree.GetListOfLeaves().At(2).GetValue(1)
    jet2Px = tree.GetListOfLeaves().At(3).GetValue(0)
    jet2Py = tree.GetListOfLeaves().At(3).GetValue(1)
    
    nArr[0] = -ePx - jet1Px - jet2Px
    nArr[1] = -ePy - jet1Px - jet2Px
    nArr[2] = r.minimization_func(w2Arr, eArr)
    nArr[3] = r.TMath.Sqrt(nArr[0]**2 + nArr[1]**2 + nArr[2]**2)
    
    tree.GetBranch("neutrino").Fill() # Filling just "neutrino" branch

 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 pTotSqEl     2.32044e+02  1.00000e+00     no limits
     2 pTSqEl       1.67332e+03  1.00000e+00     no limits
     3 ePz         -3.79641e+01  1.00000e+00     no limits
     4 nPz          3.44715e+01  5.00000e-02     no limits
 **********
 **    1 **FIX           1
 **********
 **********
 **    2 **FIX           2
 **********
 **********
 **    3 **FIX           3
 **********
 **********
 **    4 **MIGRAD 
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-04
 FCN=3.18318 FROM MIGRAD    STATUS=INITIATE        4 CALLS           5 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUESS       STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  pTotSqEl     2.32044e+02     fixed    
  

In [7]:
#Scanning just for W (decaying into electron and neutrino), electron and neutrino
tree.Scan()

400

***********************************************************************************************************
*    Row   * Instance * w1Boson.w * w2Boson.w * jet1.jet1 * jet2.jet2 * electron. * beta.beta * neutrino. *
***********************************************************************************************************
*        0 *        0 * -9.093357 * 9.0933577 * -39.67839 * 30.585034 * -11.33972 * -0.100287 * 20.433077 *
*        0 *        1 * -2.005479 * 2.0054793 * -18.17224 * 16.166761 * 10.171273 *         0 * -1.077916 *
*        0 *        2 * -11.82030 * -4.492571 * -16.76515 * 4.9448492 * -37.96411 *         0 * 36.345696 *
*        0 *        3 * 81.697739 * 80.962857 * 46.751208 * 34.946531 * 40.906210 *         0 * 41.709498 *
*        1 *        0 * -2.739752 * 2.7397526 * -11.40793 * 8.6681785 * 37.221931 * 0.0860848 * -34.48217 *
*        1 *        1 * 7.8794071 * -7.879407 * 28.564535 * -20.68512 * -2.388246 *         0 * 5.1279987 *
*        1 *        2 * 1.45

Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> 